# Zep Dining Assistant

This notebook demonstrates a personal dining assistant agent built with Zep AI's graph-based memory and the Strands agent framework. The demo walks through a realistic user scenario, showcasing how the agent leverages semantic, episodic, and procedural memory to provide personalized dining recommendations and manage user preferences over time.

## 📦 Package Installation

This section installs all required dependencies for the demo, including the Strands agent framework, Zep AI memory client, and libraries for data handling and visualization. These packages enable the agent's core logic, memory management, and graph visualization features.

In [1]:
!uv pip install strands-agents
!uv pip install strands-agents strands-agents-tools
!uv pip install zep-cloud
!uv pip install pandas
!uv pip install IPython
!uv pip install warnings

Audited 1 package in 8ms
Audited 2 packages in 11ms
Audited 1 package in 3ms
Audited 1 package in 5ms
Audited 1 package in 2ms
Audited 1 package in 4ms
Audited 1 package in 6ms
  × No solution found when resolving dependencies:                                  
  ╰─▶ Because warnings was not found in the package registry and you require
      warnings, we can conclude that your requirements are unsatisfiable.


## ⚙️ Environment Setup & Initialization

Here, we set up the environment by loading API keys, initializing the Zep AI client, and creating a demo user and session. This ensures that all subsequent interactions are tracked and stored in a dedicated memory graph for the demo user.

In [2]:
import os
import json
import uuid
from typing import Dict, List, Any

# Core libraries
from dotenv import load_dotenv
from zep_cloud.client import Zep
from strands import Agent, tool
import pandas as pd
from IPython.display import display, Markdown, HTML
import warnings

warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

# Initialize clients
zep_client = Zep(api_key=os.environ.get("ZEP_API_KEY"))

print("🚀 Zep Dining Assistant Demo - Setup Complete")
print("=" * 60)

# Create demo user and session for proper memory management
demo_user_id = f"demo_user_{uuid.uuid4().hex[:8]}" # The user id used previously is: demo_user_5155b519
session_id = f"session_{demo_user_id}_{uuid.uuid4().hex[:8]}"

try:
    # Create user in Zep
    user_result = zep_client.user.add(
        user_id=demo_user_id, 
        metadata={"demo": True, "name": "Demo User"}
    )
    
    # Create session for this user
    session_result = zep_client.memory.add_session(
        session_id=session_id,
        user_id=demo_user_id,
        metadata={"type": "dining_assistant_demo"}
    )
    
    print(f"✅ Zep connection verified")
    print(f"👤 Demo User: {demo_user_id}")
    print(f"📝 Session: {session_id}")
    
except Exception as e:
    print(f"❌ Zep setup failed: {e}")
    print("Please ensure ZEP_API_KEY is set in your .env file")

🚀 Zep Dining Assistant Demo - Setup Complete
✅ Zep connection verified
👤 Demo User: demo_user_5155b519
📝 Session: session_demo_user_5155b519_de8b418d


## 🔧 Mock External Services

To provide a consistent and realistic demo experience, we define mock calendar and restaurant APIs. These services simulate real-world data, such as calendar conflicts and restaurant options, allowing the agent to reason about availability and make recommendations without relying on external APIs.

In [3]:
class MockCalendarAPI:
    """Mock calendar service for realistic demo scenarios"""

    def __init__(self):
        # Predefined conflicts for consistent demo experience
        self.conflicts = {
            "2025-01-20 18:00": {
                "event": "Team Meeting",
                "duration": 60,
                "flexibility": "low",
                "alternative_times": ["2025-01-20 19:30", "2025-01-20 20:00"],
            },
            "6:00 PM": {
                "event": "Team Meeting",
                "duration": 60,
                "flexibility": "low",
                "alternative_times": ["7:30 PM", "8:00 PM"],
            },
            "18:00": {
                "event": "Team Meeting",
                "duration": 60,
                "flexibility": "low",
                "alternative_times": ["19:30", "20:00"],
            },
        }

    def check_availability(
        self, datetime_str: str, duration: int = 90
    ) -> Dict[str, Any]:
        """Check if the requested time is available"""
        # Check for conflicts with different time formats
        for conflict_time in self.conflicts:
            if conflict_time in datetime_str or datetime_str in conflict_time:
                conflict = self.conflicts[conflict_time]
                return {
                    "available": False,
                    "conflict": conflict,
                    "suggested_times": conflict["alternative_times"],
                }
        return {"available": True, "conflict": None}


class MockRestaurantAPI:
    """Mock restaurant service with realistic data"""

    def __init__(self):
        self.restaurants = [
            {
                "id": "resto_001",
                "name": "Ocean's Bounty",
                "cuisine": "seafood",
                "price_range": "$$",
                "rating": 4.5,
                "location": "Downtown",
                "distance": 0.8,
                "accepts_reservations": True,
                "specialties": ["fresh salmon", "lobster bisque", "grilled octopus"],
            },
            {
                "id": "resto_002",
                "name": "Green Garden Bistro",
                "cuisine": "vegetarian",
                "price_range": "$",
                "rating": 4.2,
                "location": "Midtown",
                "distance": 1.2,
                "accepts_reservations": True,
                "specialties": ["quinoa bowls", "veggie burgers", "fresh salads"],
            },
            {
                "id": "resto_003",
                "name": "Bella Italia",
                "cuisine": "italian",
                "price_range": "$$",
                "rating": 4.3,
                "location": "Little Italy",
                "distance": 1.5,
                "accepts_reservations": True,
                "specialties": ["handmade pasta", "wood-fired pizza", "osso buco"],
            },
            {
                "id": "resto_004",
                "name": "Sakura Sushi",
                "cuisine": "japanese",
                "price_range": "$$$",
                "rating": 4.7,
                "location": "Japantown",
                "distance": 2.1,
                "accepts_reservations": True,
                "specialties": ["omakase", "fresh sashimi", "tempura"],
            },
        ]

    def search_restaurants(self, preferences: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Search restaurants based on preferences"""
        results = []

        for restaurant in self.restaurants:
            score = 0

            # Cuisine match
            if (
                preferences.get("cuisine")
                and restaurant["cuisine"] == preferences["cuisine"]
            ):
                score += 3

            # Price range match
            if (
                preferences.get("budget")
                and restaurant["price_range"] == preferences["budget"]
            ):
                score += 2

            # Distance preference (closer is better)
            distance_score = max(0, 3 - restaurant["distance"])
            score += distance_score

            # Rating bonus
            score += restaurant["rating"] / 5 * 2

            restaurant_result = restaurant.copy()
            restaurant_result["match_score"] = score
            results.append(restaurant_result)

        # Sort by match score
        results.sort(key=lambda x: x["match_score"], reverse=True)
        return results[:3]  # Return top 3 matches

    def book_restaurant(
        self, restaurant_id: str, datetime_str: str, party_size: int
    ) -> Dict[str, Any]:
        """Mock restaurant booking"""
        import random

        # 85% success rate for realism
        if random.random() < 0.85:
            confirmation_id = f"CONF_{uuid.uuid4().hex[:8].upper()}"
            return {
                "success": True,
                "confirmation_id": confirmation_id,
                "message": f"Reservation confirmed for {party_size} people",
            }
        else:
            return {
                "success": False,
                "message": "Sorry, that time slot is no longer available",
            }


# Initialize mock services
calendar_api = MockCalendarAPI()
restaurant_api = MockRestaurantAPI()

print("📅 Mock Calendar API initialized")
print("🍽️  Mock Restaurant API initialized")
print(f"   - {len(restaurant_api.restaurants)} restaurants available")

📅 Mock Calendar API initialized
🍽️  Mock Restaurant API initialized
   - 4 restaurants available


## 🛠️ Strands Agent Tools Configuration

This section defines the agent's tools using the Strands framework. Each tool encapsulates a specific capability, such as checking calendar availability, searching for restaurants, booking reservations, and interacting with the Zep memory system. These tools are orchestrated by the agent to fulfill user requests and manage memory.

In [4]:
# Define tools using @tool decorator
@tool
def calendar_check(datetime: str, duration: int = 90) -> str:
    """Check calendar availability for a proposed dining time"""
    result = calendar_api.check_availability(datetime, duration)
    return json.dumps(result, indent=2)

@tool
def restaurant_search(preferences: Dict[str, Any]) -> str:
    """Search for restaurants based on user preferences"""
    results = restaurant_api.search_restaurants(preferences)
    return json.dumps(results, indent=2)

@tool
def restaurant_booking(restaurant_id: str, datetime: str, party_size: int) -> str:
    """Book a restaurant reservation"""
    result = restaurant_api.book_restaurant(restaurant_id, datetime, party_size)
    return json.dumps(result, indent=2)

@tool
def store_dining_memory(content: str) -> str:
    """Store dining-related information in user's memory"""
    try:
        # Add message to session to trigger memory storage
        zep_client.memory.add(
            session_id=session_id,
            messages=[{
                "role": "assistant",
                "content": f"Storing user preference: {content}"
            }]
        )
        
        return f"Successfully stored: {content}"
    except Exception as e:
        return f"Failed to store memory: {str(e)}"

@tool
def retrieve_dining_memories(query: str) -> str:
    """Retrieve relevant dining memories and preferences"""
    try:
        # Get memory context for the session
        memory = zep_client.memory.get(session_id=session_id)
        
        if memory and memory.context:
            return f"Retrieved memories: {memory.context}"
        else:
            return "No relevant memories found for this query."
            
    except Exception as e:
        return f"Failed to retrieve memories: {str(e)}"

@tool
def search_dining_facts(query: str) -> str:
    """Search for specific facts in user's dining history"""
    try:
        # Search for facts in the user's graph
        search_results = zep_client.graph.search(
            user_id=demo_user_id, 
            query=query, 
            limit=5
        )
        
        if search_results and hasattr(search_results, 'edges'):
            facts = [edge.fact for edge in search_results.edges if hasattr(edge, 'fact') and edge.fact]
            if facts:
                return f"Found {len(facts)} relevant facts: {json.dumps(facts, indent=2)}"
        
        return f"No specific facts found for query: {query}"
        
    except Exception as e:
        return f"Fact search failed: {str(e)}"

print("🔧 Strands agent tools configured:")
print("   ✅ Calendar check tool")
print("   ✅ Restaurant search tool") 
print("   ✅ Restaurant booking tool")
print("   ✅ Memory storage tool")
print("   ✅ Memory retrieval tool")
print("   ✅ Fact search tool")

🔧 Strands agent tools configured:
   ✅ Calendar check tool
   ✅ Restaurant search tool
   ✅ Restaurant booking tool
   ✅ Memory storage tool
   ✅ Memory retrieval tool
   ✅ Fact search tool


## 🤖 Dining Assistant Agent Initialization

Here, we initialize the Strands agent with a system prompt and the previously defined tools. The prompt guides the agent's behavior, ensuring it uses memory and external tools in the correct order to provide a personalized and context-aware dining experience.

In [5]:
DINING_ASSISTANT_PROMPT = f"""
You are a sophisticated personal dining assistant powered by Zep AI's memory system and the Strands framework.

Your core capabilities:
- Store and retrieve user dining preferences using memory tools
- Handle both permanent and temporary dining preferences  
- Manage scheduling conflicts intelligently
- Make personalized restaurant recommendations
- Learn from user feedback to improve future suggestions

Memory Management Guidelines:
1. ALWAYS retrieve memories first using retrieve_dining_memories() before asking questions
2. Store new preferences immediately using store_dining_memory() when you learn them
3. Use search_dining_facts() for specific factual queries about past experiences
4. Distinguish between permanent preferences (vegetarian) and temporary desires (seafood tonight)
5. ALWAYS check calendar using calendar_check() when a specific time is mentioned

Current session: {session_id}
User: {demo_user_id}

Available tools:
- retrieve_dining_memories: Get user's dining history and preferences 
- store_dining_memory: Save new preferences and experiences
- search_dining_facts: Search for specific facts in dining history
- calendar_check: Check for scheduling conflicts (ALWAYS use when time mentioned)
- restaurant_search: Find restaurants matching preferences
- restaurant_booking: Make restaurant reservations

Remember: Use memory tools to provide personalized, contextual assistance based on accumulated knowledge.

IMPORTANT: When user mentions a time, ALWAYS check calendar_check() first, then retrieve memories, then proceed.
"""

# Initialize Strands Agent
dining_agent = Agent(
    system_prompt=DINING_ASSISTANT_PROMPT,
    tools=[
        calendar_check,
        restaurant_search, 
        restaurant_booking,
        store_dining_memory,
        retrieve_dining_memories,
        search_dining_facts,
    ],
)

print("🤖 Strands Dining Assistant Agent Initialized")
print(f"👤 Demo User ID: {demo_user_id}")
print(f"📝 Session ID: {session_id}")
print("🧠 Zep Memory System: Connected")
print("=" * 60)

🤖 Strands Dining Assistant Agent Initialized
👤 Demo User ID: demo_user_5155b519
📝 Session ID: session_demo_user_5155b519_de8b418d
🧠 Zep Memory System: Connected


## 🍽️ Demo Scenario 1: Initial Preference Discovery

This scenario simulates a user's first interaction with the dining assistant. The agent checks for calendar conflicts, retrieves any existing preferences, and prompts the user for additional information to tailor its recommendations.

In [ ]:
display(
    Markdown("*User requests dinner for the first time - agent discovers preferences*")
)

# Simulate user's first request
user_message_1 = "Hi! I'd like to have dinner tonight around 6 PM. Can you help me find a good restaurant?"

print("👤 User:", user_message_1)
print("\n🤖 Assistant Response:")
print("-" * 40)

# Get agent response
response_1 = dining_agent(user_message_1)
print(response_1)

print("\n" + "=" * 60)

*User requests dinner for the first time - agent discovers preferences*

👤 User: Hi! I'd like to have dinner tonight around 6 PM. Can you help me find a good restaurant?

🤖 Assistant Response:
----------------------------------------
I'd be happy to help you find a restaurant for dinner tonight at 6 PM. First, let me check your calendar to make sure you're available, and then I'll look up your dining preferences to give you personalized recommendations.
Tool #1: calendar_check
I see there's a scheduling conflict at 6 PM - you have a Team Meeting that will last for 60 minutes. Let me check if you have any specific dining preferences before suggesting alternative times.
Tool #2: retrieve_dining_memories
I see you have a Team Meeting at 6 PM tonight, so that time won't work for dinner. Your calendar suggests 7:30 PM or 8:00 PM would be available instead.

Since I don't have any information about your dining preferences yet, could you share:
- What type of cuisine you're interested in tonight?
- Any dietary restrictions I should be aware of?
- How many people w

## 🧠 Memory State Analysis

This section inspects the current state of the user's memory after initial interactions. It displays stored context, messages, and graph edges, providing insight into how the agent accumulates and organizes knowledge over time.

In [7]:
def check_memory_state():
    """Check current memory state for debugging"""
    try:
        # Get current session memory
        memory = zep_client.memory.get(session_id=session_id)
        
        print(f"📊 Session Memory Status:")
        if memory:
            print(f"   ✅ Memory object exists")
            if hasattr(memory, 'context') and memory.context:
                print(f"   ✅ Context available: {len(memory.context)} characters")
                print(f"   📝 Context preview: {memory.context[:200]}...")
            else:
                print(f"   ⚠️ No context available yet")
                
            if hasattr(memory, 'messages') and memory.messages:
                print(f"   💬 Messages: {len(memory.messages)} stored")
            else:
                print(f"   💬 No messages stored yet")
        else:
            print(f"   ❌ No memory found for session")
            
        # Try to get user's graph data
        try:
            search_results = zep_client.graph.search(
                user_id=demo_user_id,
                query="dining preferences",
                limit=5
            )
            if search_results and hasattr(search_results, 'edges'):
                print(f"   🔗 Graph edges found: {len(search_results.edges)}")
            else:
                print(f"   🔗 No graph edges found yet")
        except Exception as e:
            print(f"   🔗 Graph search not ready: {str(e)}")
            
    except Exception as e:
        print(f"❌ Memory check failed: {e}")

In [ ]:
# Check memory after each interaction
check_memory_state()

📊 Session Memory Status:
   ✅ Memory object exists
   ✅ Context available: 2234 characters
   📝 Context preview: 
FACTS and ENTITIES represent relevant context to the current conversation.

# These are the most relevant facts and their valid date ranges
# format: FACT (Date range: from - to)
<FACTS>
  - On July ...
   💬 Messages: 3 stored
   🔗 Graph edges found: 4


## 📅 Demo Scenario 2: Calendar Conflict & Resolution

The agent detects a scheduling conflict for the requested dinner time and negotiates an alternative. It also stores the user's dietary and budget preferences, demonstrating how procedural and episodic memory are updated in response to user input.

In [ ]:
display(Markdown("*Agent discovers calendar conflict and negotiates alternative time*"))

# Follow up with more specific preferences and a time that has a conflict
user_message_2 = "I'm vegetarian and prefer places that aren't too expensive. Actually, can we make it exactly at 6:00 PM? That would be perfect."

print("👤 User:", user_message_2)
print("\n🤖 Assistant Response:")
print("-" * 40)

response_2 = dining_agent(user_message_2)
print(response_2)

print("\n" + "=" * 60)

# Check if memory was saved properly
print("\n🧠 Checking Memory Storage:")
try:
    # Search for newly stored preferences
    memory_check = zep_client.graph.search(
        user_id=demo_user_id, query="vegetarian preferences", limit=5
    )

    if memory_check and hasattr(memory_check, "edges") and memory_check.edges:
        print(f"✅ Found {len(memory_check.edges)} stored memories")
        for edge in memory_check.edges:
            if hasattr(edge, "fact") and edge.fact:
                print(f"   • {edge.fact}")
    else:
        print("⚠️ No memories found yet - may take a moment to process")

except Exception as e:
    print(f"Memory check error: {e}")

*Agent discovers calendar conflict and negotiates alternative time*

👤 User: I'm vegetarian and prefer places that aren't too expensive. Actually, can we make it exactly at 6:00 PM? That would be perfect.

🤖 Assistant Response:
----------------------------------------
Thank you for sharing your dietary preference as vegetarian and that you prefer places that aren't too expensive. Let me check again about the 6:00 PM timeframe since you specifically requested it.
Tool #3: calendar_check
I need to let you know that you still have a Team Meeting at 6:00 PM that appears to be inflexible. According to your calendar, the earliest available times after your meeting would be 7:30 PM or 8:00 PM.

Let me store your dining preferences in my memory so I can provide better recommendations in the future.
Tool #4: store_dining_memory
Unfortunately, 6:00 PM won't work for dinner tonight due to your Team Meeting. Would you prefer a 7:30 PM reservation instead? Also, may I ask how many people would be joining for dinner? Once you confirm a time that works with your sched

## 🔄 Demo Scenario 3: Preference Evolution

The user temporarily changes their preference from vegetarian to seafood. The agent distinguishes between permanent and temporary preferences, updating the memory graph accordingly to reflect this context-specific change.

In [ ]:
display(
    Markdown(
        "*User changes preference temporarily - showcasing temporal vs permanent preferences*"
    )
)

user_message_3 = "You know what, I'm actually really craving seafood tonight instead. I know I'm usually vegetarian, but I'm in the mood for something different tonight."

print("👤 User:", user_message_3)
print("\n🤖 Assistant Response:")
print("-" * 40)

response_3 = dining_agent(user_message_3)
print(response_3)

print("\n" + "=" * 60)

*User changes preference temporarily - showcasing temporal vs permanent preferences*

👤 User: You know what, I'm actually really craving seafood tonight instead. I know I'm usually vegetarian, but I'm in the mood for something different tonight.

🤖 Assistant Response:
----------------------------------------
I understand you're craving seafood tonight even though you typically follow a vegetarian diet. That's perfectly fine - let me update my understanding of your preferences for tonight.
Tool #5: store_dining_memory
Now, let me search for seafood restaurants that aren't too expensive. But first, I need to confirm a time that works with your schedule and how many people will be dining.

Based on your calendar availability, would 7:30 PM work for your seafood dinner tonight? And how many people should I include in the reservation?Now, let me search for seafood restaurants that aren't too expensive. But first, I need to confirm a time that works with your schedule and how many people will be dining.

Based on your calendar availability, would 7:30 PM work for your seafood

## 🎯 Demo Scenario 4: Restaurant Search & Booking

The agent uses the accumulated preferences and resolved schedule to search for suitable restaurants and book a reservation. This demonstrates the full workflow from preference discovery to action, with all steps informed by the memory graph.

In [ ]:
display(
    Markdown(
        "*Agent finds and books restaurant based on memory and current preferences*"
    )
)

user_message_4 = (
    "Great! Let's go with Ocean's Bounty for 7:30 PM. Can you book it for 2 people?"
)

print("👤 User:", user_message_4)
print("\n🤖 Assistant Response:")
print("-" * 40)

response_4 = dining_agent(user_message_4)
print(response_4)

print("\n" + "=" * 60)

*Agent finds and books restaurant based on memory and current preferences*

👤 User: Great! Let's go with Ocean's Bounty for 7:30 PM. Can you book it for 2 people?

🤖 Assistant Response:
----------------------------------------
I'll help you book a reservation at Ocean's Bounty for 7:30 PM for 2 people. Let me first confirm this time is still available in your calendar, and then I'll make the booking.
Tool #6: calendar_check
Great! 7:30 PM is available in your calendar. Now I'll book your reservation at Ocean's Bounty for 2 people.
Tool #7: restaurant_booking

Tool #8: store_dining_memory
Your reservation has been successfully booked! 

Here are the details:
- Restaurant: Ocean's Bounty
- Date and time: Tonight (July 6, 2023) at 7:30 PM
- Party size: 2 people
- Confirmation ID: CONF_F39A36A8

Your reservation is all set. I hope you enjoy your seafood dinner tonight! Is there anything else you need help with regarding your reservation or any other dining plans?Your reservation has been successfully booked! 

Here are the details:
- Restaurant: Ocean's Bounty
- D

## 🧠 Complete Memory Graph Analysis

This section analyzes the complete memory graph built during the conversation. It summarizes permanent and temporary preferences, past experiences, and the structure of the knowledge graph, highlighting the agent's ability to learn and adapt.

In [ ]:
display(Markdown("*Examining the full knowledge graph built through our conversation*"))

try:
    # Get all edges (facts) for this user
    all_edges = zep_client.graph.edge.get_by_user_id(user_id=demo_user_id)
    
    print(f"📊 Total Memory Entries: {len(all_edges)}")
    print("\n🔍 Discovered Knowledge:")

    permanent_prefs = []
    temporary_prefs = []
    experiences = []

    for edge in all_edges:
        fact = edge.fact.lower()
        if "vegetarian" in fact and "usually" in fact:
            permanent_prefs.append(edge.fact)
        elif "seafood" in fact and ("tonight" in fact or "craving" in fact):
            temporary_prefs.append(edge.fact)
        elif "booked" in fact or "reservation" in fact:
            experiences.append(edge.fact)
        else:
            permanent_prefs.append(edge.fact)

    print(f"\n🎯 Permanent Preferences ({len(permanent_prefs)}):")
    for pref in permanent_prefs:
        print(f"   • {pref}")

    print(f"\n⏰ Temporary Preferences ({len(temporary_prefs)}):")
    for pref in temporary_prefs:
        print(f"   • {pref}")

    print(f"\n📝 Experiences ({len(experiences)}):")
    for exp in experiences:
        print(f"   • {exp}")

    # Get graph structure using nodes
    try:
        user_nodes = zep_client.graph.node.get_by_user_id(demo_user_id, limit=30)
        print(f"\n🔗 Graph Structure: {len(user_nodes)} nodes")

        # Display node information
        print("   Node Details:")
        for i, node in enumerate(user_nodes[:10]):  # Show first 10 nodes
            node_name = getattr(node, 'name', 'Unknown')
            node_label = getattr(node, 'labels', ['Unknown'])
            print(f"   • Node {i+1}: {node_name} ({node_label})")
            
        if len(user_nodes) > 10:
            print(f"   • ... and {len(user_nodes) - 10} more nodes")

    except Exception as e:
        print(f"Graph structure analysis: {e}")
        
    # Also try targeted searches for specific memory types
    print(f"\n🔍 Additional Memory Searches:")
    
    try:
        # Search for dining preferences
        dining_memories = zep_client.graph.search(
            user_id=demo_user_id,
            query="dining preferences vegetarian seafood restaurant",
            limit=10
        )
        print(f"   • Dining-related memories: {len(dining_memories.edges)}")
        
        # Search for booking experiences  
        booking_memories = zep_client.graph.search(
            user_id=demo_user_id,
            query="booking reservation restaurant confirmed",
            limit=10
        )
        print(f"   • Booking experiences: {len(booking_memories.edges)}")
        
    except Exception as e:
        print(f"   • Targeted search error: {e}")

    # Memory Graph Insights
    print(f"\n💡 Memory Graph Insights:")
    print(f"   • Total facts stored: {len(all_edges)}")
    print(f"   • Graph nodes created: {len(user_nodes) if 'user_nodes' in locals() else 'N/A'}")
    print(f"   • Preference hierarchy maintained: Permanent vs Temporary")
    print(f"   • Experience tracking: Restaurant bookings with context")
    print(f"   • Temporal awareness: Date-specific preferences captured")

except Exception as e:
    print(f"Memory analysis error: {e}")

*Examining the full knowledge graph built through our conversation*

📊 Total Memory Entries: 4

🔍 Discovered Knowledge:

🎯 Permanent Preferences (2):
   • User prefers restaurants that aren't too expensive
   • User is vegetarian

⏰ Temporary Preferences (1):
   • On July 6, 2023, user specifically mentioned craving seafood as a temporary preference.

📝 Experiences (1):
   • On July 6, 2023, user made a reservation at Ocean's Bounty seafood restaurant for 7:30 PM for 2 people.

🔗 Graph Structure: 6 nodes
   Node Details:
   • Node 1: vegetarian preference (['Entity', 'Preference'])
   • Node 2: assistant (norole) (['Entity', 'Preference'])
   • Node 3: user (['Entity', 'User'])
   • Node 4: seafood preference (['Entity', 'Preference'])
   • Node 5: Ocean's Bounty seafood restaurant (['Entity'])
   • Node 6: assistant (['Entity'])

🔍 Additional Memory Searches:
   • Dining-related memories: 4
   • Booking experiences: 4

💡 Memory Graph Insights:
   • Total facts stored: 4
   • Graph nodes created: 6
   • Preference hierarchy maintained: Permanent vs Temp

## 📊 Memory Graph Visualization

A visual representation of the Zep AI knowledge graph, showing how user preferences, experiences, and relationships are interconnected. This helps illustrate the advantages of graph-based memory over traditional flat storage.

# Zep Memory Graph
This is the User Relationship Graph, a visualization of user relationships and connections from Zep
![zep memory graph](./img/zep-memory-graph.png)

## 🚀 Future Interaction Simulation

This scenario simulates a follow-up conversation days later. The agent leverages accumulated memory to provide recommendations that reflect both long-term and recent user preferences, demonstrating memory persistence and learning.

In [ ]:
display(Markdown("*Simulating a conversation days later to show memory persistence*"))

print("⏰ Simulating conversation 3 days later...")
print("=" * 60)

user_message_future = "Hi! I'm looking for a dinner recommendation for this Friday. What do you suggest based on what you know about me?"

print("👤 User:", user_message_future)
print("\n🤖 Assistant Response:")
print("-" * 40)

response_future = dining_agent(user_message_future)
print(response_future)

print("\n" + "=" * 60)

*Simulating a conversation days later to show memory persistence*

⏰ Simulating conversation 3 days later...
👤 User: Hi! I'm looking for a dinner recommendation for this Friday. What do you suggest based on what you know about me?

🤖 Assistant Response:
----------------------------------------
I'd be happy to provide dinner recommendations for this Friday based on what I know about you. Let me first check what dining preferences and history I have on record for you.
Tool #9: retrieve_dining_memories
Based on the information I have about your dining preferences, I know:

1. You're generally vegetarian but occasionally make exceptions (like your seafood craving recently)
2. You prefer restaurants that aren't too expensive
3. You recently dined at Ocean's Bounty seafood restaurant

Before I make a recommendation for this Friday, let me check your calendar to make sure there are no conflicts:
Tool #10: calendar_check
Great! It looks like you're available around 7 PM this Friday. Now let me search for restaurants that would suit your preferences:
Tool #11:

## 📈 Performance Metrics & Demo Insights

This section highlights the strengths of the Zep + Strands integration.

---

💪 Strengths of Zep:

- ✅ Memory Persistence: Preferences stored across conversation
- ✅ Temporal Understanding: Temporary vs permanent preferences handled
- ✅ Conflict Resolution: Calendar conflicts detected and resolved
- ✅ Personalization: Recommendations based on stored preferences
- ✅ Learning: Agent learns from each interaction
- ✅ Tool Integration: Seamless Strands tool orchestration

🧠 Key Differentiators Demonstrated:

- ✅ Graph-based memory with relationships
- ✅ Temporal preference handling
- ✅ Context-aware recommendations
- ✅ Multi-turn conversation memory
- ✅ Real-time learning and adaptation

In [ ]:
print("=" * 50)

print("\n📊 Demo Statistics:")
print(f"   • User ID: {demo_user_id}")
print(f"   • Session ID: {session_id}")
print("   • Interactions: 5 conversations")
print("   • Tools Used: Calendar, Restaurant Search, Booking, Memory")
print("   • Memory Entries: Multiple preferences and experiences stored")
print("   • Conflicts Resolved: 1 (6 PM → 7:30 PM)")
print("   • Bookings Made: 1 (Ocean's Bounty)")

📊 Demo Statistics:
   • User ID: demo_user_5155b519
   • Session ID: session_demo_user_5155b519_de8b418d
   • Interactions: 5 conversations
   • Tools Used: Calendar, Restaurant Search, Booking, Memory
   • Memory Entries: Multiple preferences and experiences stored
   • Conflicts Resolved: 1 (6 PM → 7:30 PM)
   • Bookings Made: 1 (Ocean's Bounty)


## 🚀 Next Steps & Future Possibilities

**🔮 Potential Enhancements:**
-   🌐 Integration with real restaurant APIs (Yelp, OpenTable)
-   📱 Multi-channel support (SMS, voice, web app)
-   👥 Group dining coordination and preference merging
-   🎂 Special occasion and celebration planning
-   🏃 Activity-based dining suggestions (post-workout, business lunch)
-   🌍 Travel dining recommendations with location memory
-   📊 Analytics dashboard for dining pattern insights
-   🤖 Multi-agent collaboration (calendar agent, nutrition agent)

**✨ Business Value:**
-    Reduced decision fatigue for users
-    Increased restaurant booking conversion
-    Higher user engagement through personalization
-    Valuable dining behavior insights

🎉 Demo Complete!
============================================================
Thank you for exploring the Zep + Strands Dining Assistant!
This POC demonstrates the power of graph-based memory for creating
more personalized and intelligent AI agents.